In [1]:
from sklearn import svm
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

%matplotlib inline
train_data = pd.read_csv("Data/CE802_P2_Data.csv",sep=',')
test_data = pd.read_csv("Data/CE802_P2_Test.csv",sep=',')
file_name="Data/CE802_P2_Test_SVM.csv"


Bad key "text.kerning_factor" on line 4 in
C:\Conda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
train_data

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,Class
0,-471.81,-1.14,20,7.45,1.98,-0.18,-9.18,-38.61,-28.14,-12.90,60,-189.87,8.45,4.85,9.24,False
1,-546.81,-8.52,20,6.40,4.24,3.42,-11.31,-46.26,-48.14,-11.28,350,-44.87,7.20,2.91,10.32,False
2,-486.81,-6.66,20,10.15,-1.22,3.34,-14.01,-39.81,-48.14,-12.14,190,-124.87,8.55,6.31,11.25,True
3,-282.81,8.37,2,0.38,3.58,-16.24,2.19,-22.26,-22.14,-22.32,10,-34.87,1.43,8.50,10.15,False
4,-336.81,-2.16,20,8.45,-1.18,2.94,-11.10,-37.11,-28.14,-15.84,105,-124.87,6.45,4.17,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,-366.81,4.20,20,6.40,1.92,1.16,-4.98,-39.81,-48.14,-14.44,335,-209.87,9.05,6.70,9.00,False
1496,-186.81,2.94,20,5.35,-0.84,-10.00,-6.51,-37.11,-48.14,-18.18,95,-39.87,10.45,4.86,9.31,False
1497,-240.81,7.92,2,0.54,1.88,-7.66,0.78,-23.31,-22.14,-20.54,17,-42.87,0.10,8.92,NaN,False
1498,-258.81,8.31,2,0.59,-0.80,-6.64,1.95,-21.57,-26.14,-22.94,15,-41.87,1.68,9.11,10.92,False


In [3]:
test_data

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,Class
0,-234.81,9.00,0,0.40,6.70,-6.82,11.82,-21.15,-22.14,-23.18,26,-14.87,1.03,8.35,NaN,NaN
1,23.19,-6.33,20,6.85,-2.52,0.14,-13.23,-38.91,-68.14,-16.52,365,-39.87,8.30,4.55,11.64,NaN
2,-243.81,20.52,2,0.53,1.66,-8.06,2.01,-20.55,-26.14,-21.62,4,0.13,1.39,8.67,12.84,NaN
3,-306.81,-1.95,20,5.25,2.58,-0.08,-9.66,-39.51,-28.14,-18.46,100,-119.87,7.95,4.62,9.13,NaN
4,-81.81,8.19,20,5.35,-3.66,-1.84,10.32,-37.26,-48.14,-17.96,210,-189.87,12.80,6.19,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,-321.81,-7.17,20,7.25,2.14,4.84,-13.05,-44.31,-28.14,-10.76,360,-34.87,7.90,3.36,11.22,NaN
1496,-306.81,-5.28,20,7.10,0.22,2.70,-10.38,-39.81,-28.14,-16.82,295,-64.87,14.05,3.45,NaN,NaN
1497,-270.81,8.10,2,0.88,2.58,-7.40,3.87,-21.48,-26.14,-20.74,20,-33.87,1.29,8.97,9.61,NaN
1498,-516.81,-2.88,20,7.60,-0.02,0.88,-7.98,-37.71,-48.14,-21.10,130,-219.87,6.65,4.86,NaN,NaN


In [4]:
def clean_data(data):
    for i in range(1,data.shape[1]):
#         false_mean=data[data["Class"]== False]["F"+str(i)].mean()
#         true_mean=data[data["Class"]== True]["F"+str(i)].mean()
        col_mean=data["F"+str(i)].mean()
        data['F'+str(i)] = data['F'+str(i)].replace(np.nan, col_mean)
    return data
    

In [5]:
train_data = clean_data(train_data)
train_data

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,Class
0,-471.81,-1.14,20,7.45,1.98,-0.18,-9.18,-38.61,-28.14,-12.90,60,-189.87,8.45,4.85,9.240000,False
1,-546.81,-8.52,20,6.40,4.24,3.42,-11.31,-46.26,-48.14,-11.28,350,-44.87,7.20,2.91,10.320000,False
2,-486.81,-6.66,20,10.15,-1.22,3.34,-14.01,-39.81,-48.14,-12.14,190,-124.87,8.55,6.31,11.250000,True
3,-282.81,8.37,2,0.38,3.58,-16.24,2.19,-22.26,-22.14,-22.32,10,-34.87,1.43,8.50,10.150000,False
4,-336.81,-2.16,20,8.45,-1.18,2.94,-11.10,-37.11,-28.14,-15.84,105,-124.87,6.45,4.17,10.743013,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,-366.81,4.20,20,6.40,1.92,1.16,-4.98,-39.81,-48.14,-14.44,335,-209.87,9.05,6.70,9.000000,False
1496,-186.81,2.94,20,5.35,-0.84,-10.00,-6.51,-37.11,-48.14,-18.18,95,-39.87,10.45,4.86,9.310000,False
1497,-240.81,7.92,2,0.54,1.88,-7.66,0.78,-23.31,-22.14,-20.54,17,-42.87,0.10,8.92,10.743013,False
1498,-258.81,8.31,2,0.59,-0.80,-6.64,1.95,-21.57,-26.14,-22.94,15,-41.87,1.68,9.11,10.920000,False


In [6]:
test_data = clean_data(test_data)
test_data

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,Class
0,-234.81,9.00,0,0.40,6.70,-6.82,11.82,-21.15,-22.14,-23.18,26,-14.87,1.03,8.35,10.776347,NaN
1,23.19,-6.33,20,6.85,-2.52,0.14,-13.23,-38.91,-68.14,-16.52,365,-39.87,8.30,4.55,11.640000,NaN
2,-243.81,20.52,2,0.53,1.66,-8.06,2.01,-20.55,-26.14,-21.62,4,0.13,1.39,8.67,12.840000,NaN
3,-306.81,-1.95,20,5.25,2.58,-0.08,-9.66,-39.51,-28.14,-18.46,100,-119.87,7.95,4.62,9.130000,NaN
4,-81.81,8.19,20,5.35,-3.66,-1.84,10.32,-37.26,-48.14,-17.96,210,-189.87,12.80,6.19,10.776347,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,-321.81,-7.17,20,7.25,2.14,4.84,-13.05,-44.31,-28.14,-10.76,360,-34.87,7.90,3.36,11.220000,NaN
1496,-306.81,-5.28,20,7.10,0.22,2.70,-10.38,-39.81,-28.14,-16.82,295,-64.87,14.05,3.45,10.776347,NaN
1497,-270.81,8.10,2,0.88,2.58,-7.40,3.87,-21.48,-26.14,-20.74,20,-33.87,1.29,8.97,9.610000,NaN
1498,-516.81,-2.88,20,7.60,-0.02,0.88,-7.98,-37.71,-48.14,-21.10,130,-219.87,6.65,4.86,10.776347,NaN


In [7]:
X = train_data.drop('Class', axis=1)
y = train_data['Class']

X_test=test_data.drop('Class',axis=1)

In [8]:
scaler = StandardScaler()
X=scaler.fit_transform(X)
X_test=scaler.fit_transform(X_test)

In [9]:
X_train, X_valid, y_train, y_valid =train_test_split(X, y, test_size = 0.20)

In [10]:
svclassifier = SVC(kernel='rbf', C=1e2, gamma=0.03)

In [11]:
svclassifier.fit(X_train, y_train)

SVC(C=100.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.03, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [12]:
cv_score = cross_val_score(svclassifier, X, y, cv = 5, scoring='f1')
print("CV mean score: ", cv_score.mean())

CV mean score:  0.8860317304664704


In [13]:
y_pred = svclassifier.predict(X_valid)

In [14]:
print(confusion_matrix(y_valid,y_pred))
print(classification_report(y_valid,y_pred))

[[150  16]
 [ 15 119]]
              precision    recall  f1-score   support

       False       0.91      0.90      0.91       166
        True       0.88      0.89      0.88       134

    accuracy                           0.90       300
   macro avg       0.90      0.90      0.90       300
weighted avg       0.90      0.90      0.90       300



In [15]:
y_pred_test = svclassifier.predict(X_test)
y_pred_test
for i in range(0,len(y_pred_test)):
    test_data.loc[i,'Class']=y_pred_test[i]


In [16]:
test_data

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,Class
0,-234.81,9.00,0,0.40,6.70,-6.82,11.82,-21.15,-22.14,-23.18,26,-14.87,1.03,8.35,10.776347,False
1,23.19,-6.33,20,6.85,-2.52,0.14,-13.23,-38.91,-68.14,-16.52,365,-39.87,8.30,4.55,11.640000,True
2,-243.81,20.52,2,0.53,1.66,-8.06,2.01,-20.55,-26.14,-21.62,4,0.13,1.39,8.67,12.840000,True
3,-306.81,-1.95,20,5.25,2.58,-0.08,-9.66,-39.51,-28.14,-18.46,100,-119.87,7.95,4.62,9.130000,False
4,-81.81,8.19,20,5.35,-3.66,-1.84,10.32,-37.26,-48.14,-17.96,210,-189.87,12.80,6.19,10.776347,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,-321.81,-7.17,20,7.25,2.14,4.84,-13.05,-44.31,-28.14,-10.76,360,-34.87,7.90,3.36,11.220000,False
1496,-306.81,-5.28,20,7.10,0.22,2.70,-10.38,-39.81,-28.14,-16.82,295,-64.87,14.05,3.45,10.776347,False
1497,-270.81,8.10,2,0.88,2.58,-7.40,3.87,-21.48,-26.14,-20.74,20,-33.87,1.29,8.97,9.610000,False
1498,-516.81,-2.88,20,7.60,-0.02,0.88,-7.98,-37.71,-48.14,-21.10,130,-219.87,6.65,4.86,10.776347,False


In [17]:
test_data.to_csv(file_name, sep=',')

In [18]:
svclassifier = SVC(kernel='linear')

In [19]:
svclassifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [20]:
cv_score = cross_val_score(svclassifier, X, y, cv = 5, scoring='f1')
print("CV mean score: ", cv_score.mean())

CV mean score:  0.8347342958463478


In [21]:
y_pred = svclassifier.predict(X_valid)

In [22]:
print(confusion_matrix(y_valid, y_pred))
print(classification_report(y_valid, y_pred))

[[140  26]
 [ 24 110]]
              precision    recall  f1-score   support

       False       0.85      0.84      0.85       166
        True       0.81      0.82      0.81       134

    accuracy                           0.83       300
   macro avg       0.83      0.83      0.83       300
weighted avg       0.83      0.83      0.83       300



https://scikit-learn.org/stable/modules/svm.html#regression
    https://stackabuse.com/implementing-svm-and-kernel-svm-with-pythons-scikit-learn/
        https://appdividend.com/2020/04/28/python-how-to-select-rows-from-pandas-dataframe/
            https://datatofish.com/replace-nan-values-with-zeros/
                https://keytodatascience.com/selecting-rows-conditions-pandas-dataframe/
                    https://appdividend.com/2020/08/06/how-to-find-mean-in-pandas-dataframe/
                        https://stackoverflow.com/questions/47953338/how-to-replace-values-of-selected-row-of-a-column-in-pandas-dataframe
                            https://www.analyticsvidhya.com/blog/2018/03/introduction-k-neighbours-algorithm-clustering/
                                https://stackoverflow.com/questions/47953338/how-to-replace-values-of-selected-row-of-a-column-in-pandas-dataframe